# Отчёт 27.11

## ТЗ

В проекте работаю над моделированием системы хранения данных. В качестве целевого значения рассматривается производительность системы, целевое значение зависит от признаков: 

**непрерывные:**

`nodes` - количество узлов: $4,\,5,\,\cdots,100$;

`size` - размер объекта: $4,\,5,\;\cdots\,128e3$ (KB); 

**категориальные:**

`type` - тип действия: запись / чтение; 

`protection` - защита: REP 2, REP 3, EC2.1;

`Hardware Chassis`: X205 / VEGMAN; 

Мы получаем данные со стенда с шестнадцатью узлами (имеем замеры для nodes $= 4, \cdots\,16$ ) для различных комбинаций оставшихся признаков. Таким образом, математически, **замер** можно определить следующим образом $$(\,\boldsymbol{v}_{i_0,i_1,i_2,i_3,i_4}\;|\;nodes(i_0),\,size(i_1),\,type(i_2),\,protection(i_3),\,HW\_chassis(i_4)\,),\;\boldsymbol{v}_{i_0,i_1,i_2,i_3,i_4}\in R$$

На имеющихся данных необходимо обучить модель $M(nodes, size, type, protection, HW\_chassis)$, в первую очередь нас интересует экстраполяционная способность этой модели по параметру $nodes$: в данных $nodes \in [4,16]$, хотим знать поведение целевого значения для $nodes \in [4, 100]$ и всевозможных комбинаций остальных признаков.

## РЕАЛИЗАЦИЯ

**Чтение замеров**

Данные (замеры) поступают в виде .txt / .md файлов, в зависимости от политики защиты (признак protection) вид записи в файл может отличаться. 

Для чтения данных написан скрипт `READER`, читающий все файлы папки sizes и записывающий необходимые данные в словарь `Data`, в котором ключи --- параметры замера. `Data` имеет следующую структуру: $$\left\{\begin{pmatrix}protection \\ size \\ nodes \\ w/r \\ HW\_chassis \end{pmatrix}:\;\begin{Bmatrix}MB/s: & list \\ avg: & list\\ min: & list \\ med: & list \\ max: & list \\ p(90): & list \\ p(95): & list \\ op/s: & list \\ op/s\_loss: & list \\ SUM\_MB/s: & float \\ SUM\_op/s: & float\end{Bmatrix}\right\}$$

*принцип работы:*

С помощью `glob.glob` собираем все .txt и .md файлы из папки, конкатенируем их.

Проходимся по файлам циклом

Обрабатываем файл так: создаём list, добавляем туда все строки из файла и с while проходимся по строчкам: мы знаем, что каждый замер начинается с '###' и имеет фиксированный тип, по ходу записываем признаки в ключ временного словаря `Params`, а целевое значение и побочную информацию (min, max, avg, et c.) во временный словарь `Input`. Так, после обработки замера нам нужно лишь присвоить `Data[tuple(Params.values())] = Input`.

**Выгрузка данных**

В ходе работы мне была поставлена задача конвертировать исходные .txt / .md данные в .xlsx документ.

Скрипты `CONVERTER` и `WRITER` конвертируют словарь `Data` в список и выгружает его в виде .xlsx файла

*принцип работы:*

`CONVERTER` циклом переписывает значения из словаря в новый список.

`WRITER` переводит list `Data` полученный в результате работы конвертера в `pd.DataFrame` и методом `.to_excel` выгружает .xlsx док.

**Отрисовка данных**

Как для сырых данных так и для предсказанных выводятся графики - скрипты `GRAPHS (RAW DATA)`, `GRAPHS (PREDICTED DATA)`. Переменная `parametr` определяет, какой признак будет по оси абсцисс - он принимает значения `size`, `nodes`. Словарь `fix_params` - фиксирует оставшиеся признаки.

**Препроцессинг данных**

Перед обучением модели необходимо предобработать данные:

Для категориальных признаков выбран метод `One Hot Encoder`.

**One Hot Encoder**

Для реализации написана функция `features_transformation(Data: dict) -> pd.DataFrame:`. Encoding реализован pd методом `.get_dummies`, в связи с этим есть некоторые проблемы с требованием ко входным данным. Поскольку мы планируем использовать кодированные данные для обучения, то их размерность должна быть *чётко фиксирована*: при использовании `M.predict()` мы должны иметь возможность посмотреть на предсказанные значения на любом наборе параметров. `.get_dummies` создаёт лишь те столбцы значений cat. признаков, которые есть в `Data`, иначе говоря, если мы выберем подсловарь в `Data`, то, возможно, количество столбцов при действии на этот подсловарь функцией `features_transformation` будет меньше чем при действии на исходный словарь и это большая проблема, ведь признаки мы хотим фиксировать, чтобы посмотреть на предсказание, и, поэтому, кол - во столбцов будет строго меньше.
 
Чтобы решить эту проблему написан скрипт `DOCUMENTATION` он фиксирует в словаре `doc` какие бывают значения параметров и добавляет в `Data` записи, перебирающие всевозможные комбинации признаков. Функция `features_transformation` обрабатывает это и выдаёт лишь DataFrame, соответствующий входному словарю. Это решает проблему зависимости размерности признаков при действии функции от входного словаря.

**Преобразование непрерывных признаков**

Знаем, что $\text{target value}\;|\;\text{nodes}\sim \log (\text{nodes})$


Введём для `numerical` параметров - `nodes, size` - базисные функции:

$$\boldsymbol{\phi}_{size}(s) = \begin{pmatrix}
e^{-(s - 8)^2 / 2e4}\\
e^{-(s - 128)^2 / 2e4}\\
e^{-(s - 1e3)^2 / 2e4}\\
e^{-(s - 128e3)^2 / 2e4}\\
1\\
s\\
e^{-s}\\
e^{-s^2}
\end{pmatrix}\in R^{8};\;\boldsymbol{\phi}_{nodes}(n) = \begin{pmatrix}
\log(n)\\
1\\
n
\end{pmatrix}\in R^3$$

$$\boldsymbol{\phi}(n,\,s) = (\;\boldsymbol{\phi}_{nodes}(n)_i\cdot \boldsymbol{\phi}_{size}(s)_j\;)_{(i,j)}\in R^{24}$$

Для `categorial` параметров - `protection, type, HW_chassis` - One Hot Encoder


Таким образом, модель приняла вид:

$$\hat{v}\,(i_0\,i_1\,\cdots\,i_4) = \boldsymbol{w}^\top \begin{pmatrix}
\boldsymbol{\phi}(nodes\,(i_0),\,size\,(i_1))\\
\boldsymbol{\text{OHE}}(protection\,(i_2),\;type\,(i_3),\;HW\_chassis\,(i_4))
\end{pmatrix}$$

где $\boldsymbol{\text{OHE}}$ - кодирование наших `cat.` параметров.

Далее - скейлим непрерывные признаки `sklearn.preprocessing.StandartScaler`.


**Обучение модели**

Модель - `sklearn.linear_model.LinearRegression`

Предобрабатываем данные, бутстрапим, на обучающей учим модель. Выводим матрицу весов.

*В дальнейшем планируется перед бутстрапингом разделить дату на тестовую и обучающую (пока данных мало - смысла в этом нет).*

**Оценка качества модели**

Модель оценивается метриками Mean Squared Error (MSE) и $R^2_{adj}$. Примем соглашение по интерперетации $R^2_{adj}$: 

$R^2_{adj} < 0.5$ - неудовлетворительно; $0.8 \geq R^2_{adj}> 0.5$ - удовлетворительно; $R^2_{adj} > 0.8$ - хорошо.

Использованы методы `.r2_score` и `mean_squared_error` библиотеки `sklearn.metrics` из $R^2$ выражается $R^2_{adj}$: $$R^2_{adj} = 1- (1 - R^2)\frac{n-1}{n-k-1}$$ $n$ - кол - во данных, $k$ - кол - во параметров.

**Результат**

Обученная модель показывает хорошие результаты по метрике $R^2_{adj}$: от $.81$ до $.815$. Для сравнения - `RandomForest` даёт показатель $\sim .9$ (поскольку экстраполяционная способность случайного леса околонулевая - мы его не используем).